# Project

### Deadline: 26th of January

---

## Scope & Ground Rules


### Part 1 - 50% | Scope

It's composed for `5` small assignments with guidelines. The assignment scores **evenly distributed across all questions** (each question accounts for 10% of the final score). 


**Any question please contact to me via Slack or Email.**


---

**IMPORTANT NOTES to have in mind** 

a) Code Readability is taken into account for the evaluation, so please make it simple, readable and explain your operations when necessary.

b) Make sure that the evaluater can re-run the notebook from the begining, i.e. before you delivery the assignment please go to the bar on top of your notebook -> `Kernel` -> `Restart & Run all`. Validate that all outputs are as you expect.

----


## How can I deliver the project?



**Email contact**

Please email me via `antony_costa_1995@hotmail.com` with the following subject:

`[MPPD Project] <Your Name>`


**Deliverable**

1) Notebook with the code used for both parts, 1 and 2.

2) The notebook **NAME** should follow the notation:

```
 <Your Name>_MPPD_project.ipynb
```

E.g. `AntonyCosta_MPPD_project.ipynb`

---

---

## Setup

Feel free to add any Python package as you please

---

# Part 1

## 1- Load Data

1.1- Load **house_prices_final_project.csv** to a Pandas DataFrame. You can see in `data_description.txt` file the description of each column

In [102]:
import pandas as pd 
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

In [103]:

df = pd.read_csv('data/house_prices_final_project.csv')

In [104]:
df.head(5)  

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


1.2- Print dataset total number of `observations` and `variables`

In [105]:
observations, variables = df.shape

print(f"Número de observações: {observations} | Número de variáveis: {variables}")

Número de observações: 1460 | Número de variáveis: 81


---

### Please find below the subset of columns we are going to consider for the rest of the assignment

In [106]:
columns_list = ['FullBath',
                'TotRmsAbvGrd',
                'Fireplaces',
                'GarageYrBlt',
                'GarageCars',
                'GarageArea',
                'LotFrontage',
                'WoodDeckSF',
                'OpenPorchSF',
                'SaleType',
                'SaleCondition',
                'SalePrice']

1.3- Create a new dataframe which is a subset of the origin dataframe based on the columns listed above.

In [107]:
# Criar um DataFrame com o subconjunto de colunas selecionadas
df_subset = df[columns_list]

# Exibir as primeiras linhas para confirmar
df_subset.head()

,FullBath,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,LotFrontage,WoodDeckSF,OpenPorchSF,SaleType,SaleCondition,SalePrice
0,2,8,0,2003.0,2,548,65.0,0,61,WD,Normal,208500
1,2,6,1,1976.0,2,460,80.0,298,0,WD,Normal,181500
2,2,6,1,2001.0,2,608,68.0,0,42,WD,Normal,223500
3,1,7,1,1998.0,3,642,60.0,0,35,WD,Abnorml,140000
4,2,9,1,2000.0,3,836,84.0,192,84,WD,Normal,250000


In [108]:
# Usar o info() consultar mem usage e da contagem de non null values de cada col

df_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   FullBath       1460 non-null   int64  
 1   TotRmsAbvGrd   1460 non-null   int64  
 2   Fireplaces     1460 non-null   int64  
 3   GarageYrBlt    1379 non-null   float64
 4   GarageCars     1460 non-null   int64  
 5   GarageArea     1460 non-null   int64  
 6   LotFrontage    1201 non-null   float64
 7   WoodDeckSF     1460 non-null   int64  
 8   OpenPorchSF    1460 non-null   int64  
 9   SaleType       1460 non-null   object 
 10  SaleCondition  1460 non-null   object 
 11  SalePrice      1460 non-null   int64  
dtypes: float64(2), int64(8), object(2)
memory usage: 137.0+ KB


## 2- Creating Labels

2.1- Create the `target` column based on `SalePrice`. The split should be done using the median value to create 2 new buckets. `Min->Median` bucket should have assigned the value `0` while the other bucket (`Median->Max`) value should be `1`.



Note: you are free to decide the buckets boundaries


In [109]:
df_subset.describe().T

,count,mean,std,min,25%,50%,75%,max
FullBath,1460.0,1.565068,0.550916,0.0,1.0,2.0,2.0,3.0
TotRmsAbvGrd,1460.0,6.517808,1.625393,2.0,5.0,6.0,7.0,14.0
Fireplaces,1460.0,0.613014,0.644666,0.0,0.0,1.0,1.0,3.0
GarageYrBlt,1379.0,1978.506164,24.689725,1900.0,1961.0,1980.0,2002.0,2010.0
GarageCars,1460.0,1.767123,0.747315,0.0,1.0,2.0,2.0,4.0
GarageArea,1460.0,472.980137,213.804841,0.0,334.5,480.0,576.0,1418.0
LotFrontage,1201.0,70.049958,24.284752,21.0,59.0,69.0,80.0,313.0
WoodDeckSF,1460.0,94.244521,125.338794,0.0,0.0,0.0,168.0,857.0
OpenPorchSF,1460.0,46.660274,66.256028,0.0,0.0,25.0,68.0,547.0
SalePrice,1460.0,180921.195890,79442.502883,34900.0,129975.0,163000.0,214000.0,755000.0


In [110]:

# Criar o KBinsDiscretizer para 2 bins usando a estratégia quantile
discretizer = KBinsDiscretizer(n_bins=2, encode='ordinal', strategy='quantile')

# Ajustar e transformar os valores de SalePrice
df_subset['SalesPriceBinned'] = discretizer.fit_transform(df_subset[['SalePrice']]).astype(int).flatten()

print(df_subset['SalesPriceBinned'].value_counts())


SalesPriceBinned
1    732
0    728
Name: count, dtype: int64


C:\Users\PereiraRodrigues\AppData\Local\Temp\ipykernel_6384\4048765849.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['SalesPriceBinned'] = discretizer.fit_transform(df_subset[['SalePrice']]).astype(int).flatten()


In [111]:
df_subset.sample(10)

,FullBath,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,LotFrontage,WoodDeckSF,OpenPorchSF,SaleType,SaleCondition,SalePrice,SalesPriceBinned
1338,2,8,0,2002.0,2,492,95.0,146,91,WD,Normal,200000,1
1370,1,6,0,1967.0,1,338,90.0,0,0,WD,Normal,105000,0
744,2,5,1,1993.0,2,462,41.0,96,0,WD,Normal,180000,1
633,1,6,0,1954.0,1,260,80.0,390,0,WD,Normal,139400,0
1330,2,7,1,2006.0,3,825,85.0,144,45,WD,Normal,227000,1
1278,2,7,1,2002.0,2,577,75.0,0,211,WD,Normal,237000,1
307,1,6,0,NaN,0,0,NaN,0,144,WD,Normal,89500,0
1329,2,8,1,1998.0,2,462,63.0,0,28,WD,Normal,176500,1
329,1,6,0,1920.0,1,189,60.0,0,0,WD,Normal,78000,0
607,3,8,0,1948.0,1,230,78.0,103,0,WD,Normal,225000,1


## 3- Handling Missing Values

3.1- List the amount of missing values per column

In [112]:
# Listar o número de valores ausentes por coluna
missing_values = df_subset.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
missing_values


LotFrontage    259
GarageYrBlt     81
dtype: int64

3.2- Take care of the missing values in the column `LotFrontage`

A mediana (69) está próxima à média (70.05), indicando uma distribuição aproximadamente simétrica, mas há valores altos como 313, sugerindo a possibilidade de outliers.
Abaixo é feito a verificação de outliers segundo o metodo intequartis

In [113]:
# Calculate IQR
Q1 = df_subset['LotFrontage'].quantile(0.25)
Q3 = df_subset['LotFrontage'].quantile(0.75)
IQR = Q3 - Q1

# Define the IQR boundaries
lower_bound_iqr = Q1 - 1.5 * IQR
upper_bound_iqr = Q3 + 1.5 * IQR

# Detect outliers using IQR
df_subset['LotFrontage_Outlier'] = df_subset['LotFrontage'].map(lambda x: 1 if x < lower_bound_iqr or x > upper_bound_iqr else 0)


# Count the number of outliers
outlier_count = df_subset['LotFrontage_Outlier'].sum()
outlier_info = {
    "Total Outliers": outlier_count,
    "Lower Bound": lower_bound_iqr,
    "Upper Bound": upper_bound_iqr
}

outlier_info

C:\Users\PereiraRodrigues\AppData\Local\Temp\ipykernel_6384\1046580130.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['LotFrontage_Outlier'] = df_subset['LotFrontage'].map(lambda x: 1 if x < lower_bound_iqr or x > upper_bound_iqr else 0)


{'Total Outliers': np.int64(88),
 'Lower Bound': np.float64(27.5),
 'Upper Bound': np.float64(111.5)}

Como existe cerca de 88 outliers, avancei pela  mediana que  é menos sensível a outliers

In [114]:
# Criando o imputer com a estratégia de mediana
imputer = SimpleImputer(strategy='median')

# Aplicando o imputer na coluna LotFrontage
df_subset[['LotFrontage']] = imputer.fit_transform(df_subset[['LotFrontage']])

# Verificando se há valores nulos após a imputação
missing_after_imputation = df_subset['LotFrontage'].isnull().sum()
print(f'Valores nulos após imputação: {missing_after_imputation}')

Valores nulos após imputação: 0


C:\Users\PereiraRodrigues\AppData\Local\Temp\ipykernel_6384\1389919621.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset[['LotFrontage']] = imputer.fit_transform(df_subset[['LotFrontage']])


## 4- Handling Categorical Data

4.1- Split categorical feature into a `df_categorical` dataframe

In [115]:
df_subset

,FullBath,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,LotFrontage,WoodDeckSF,OpenPorchSF,SaleType,SaleCondition,SalePrice,SalesPriceBinned,LotFrontage_Outlier
0,2,8,0,2003.0,2,548,65.0,0,61,WD,Normal,208500,1,0
1,2,6,1,1976.0,2,460,80.0,298,0,WD,Normal,181500,1,0
2,2,6,1,2001.0,2,608,68.0,0,42,WD,Normal,223500,1,0
3,1,7,1,1998.0,3,642,60.0,0,35,WD,Abnorml,140000,0,0
4,2,9,1,2000.0,3,836,84.0,192,84,WD,Normal,250000,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,2,7,1,1999.0,2,460,62.0,0,40,WD,Normal,175000,1,0
1456,2,7,2,1978.0,2,500,85.0,349,0,WD,Normal,210000,1,0
1457,2,9,2,1941.0,1,252,66.0,0,60,WD,Normal,266500,1,0
1458,1,5,0,1950.0,1,240,68.0,366,0,WD,Normal,142125,0,0


In [116]:
# Criar DataFrame apenas com as colunas categóricas
df_categorical = df_subset[df_subset.select_dtypes(include=['object', 'category']).columns.tolist()]

# Exibir as colunas categóricas selecionadas
print(df_categorical.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   SaleType       1460 non-null   object
 1   SaleCondition  1460 non-null   object
dtypes: object(2)
memory usage: 22.9+ KB
None


In [117]:
df_categorical.shape

(1460, 2)

In [118]:
df_categorical['SaleTypeCat'] = df_categorical['SaleType'].astype('category')
df_categorical['SaleConditionCat'] = df_categorical['SaleCondition'].astype('category')
df_categorical.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   SaleType          1460 non-null   object  
 1   SaleCondition     1460 non-null   object  
 2   SaleTypeCat       1460 non-null   category
 3   SaleConditionCat  1460 non-null   category
dtypes: category(2), object(2)
memory usage: 26.4+ KB


C:\Users\PereiraRodrigues\AppData\Local\Temp\ipykernel_6384\4192965614.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_categorical['SaleTypeCat'] = df_categorical['SaleType'].astype('category')
C:\Users\PereiraRodrigues\AppData\Local\Temp\ipykernel_6384\4192965614.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_categorical['SaleConditionCat'] = df_categorical['SaleCondition'].astype('category')


4.2- Apply OHE to `SaleType`

In [119]:
# Inicializar o OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)  

# Ajustar o encoder e transformar os dados
salestype_ohe = ohe.fit_transform(df_categorical[['SaleType']])

# Criar um DataFrame com os resultados do OHE
#ohe_columns = [f"SaleType_{category}" for category in ohe.categories_[0]]
ohe_df = pd.DataFrame(salestype_ohe, columns=list(ohe.categories_[0]))

# Concatenar o resultado ao DataFrame original
df_categorical = pd.concat([df_categorical, ohe_df], axis=1)

# Remover a coluna original 'SaleType' já que foi codificada
df_categorical.drop(columns=['SaleType'], inplace=True)



In [120]:
# Exibir as primeiras linhas do DataFrame atualizado para verificar o resultado
df_categorical.sample(15)

,SaleCondition,SaleTypeCat,SaleConditionCat,COD,CWD,Con,ConLD,ConLI,ConLw,New,Oth,WD
159,Partial,New,Partial,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
104,Normal,WD,Normal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
721,Normal,WD,Normal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1308,Normal,WD,Normal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1202,Normal,WD,Normal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
101,Normal,WD,Normal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
197,Abnorml,WD,Abnorml,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1180,Normal,WD,Normal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
264,Normal,WD,Normal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1437,Partial,New,Partial,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## 5- Feature Scaling

5.1- Apply feature scaling to the variable `GarageArea`. Make sure that the new range fall between `-1/3` and `3`.

In [121]:
# Apply feature scaling to the variable `GarageArea`. Make sure that the new range fall between `-1/3` and `3`.


print(df_subset['GarageArea'].describe())


count    1460.000000
mean      472.980137
std       213.804841
min         0.000000
25%       334.500000
50%       480.000000
75%       576.000000
max      1418.000000
Name: GarageArea, dtype: float64


In [122]:
# Apply feature scaling to the variable `GarageArea`. Make sure that the new range fall between `-1/3` and `3`.


mmscaler = MinMaxScaler(feature_range=(-1/3, 3))
df_subset['GarageAreaNormalized'] = mmscaler.fit_transform(df_subset[['GarageArea']])


C:\Users\PereiraRodrigues\AppData\Local\Temp\ipykernel_6384\639715783.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['GarageAreaNormalized'] = mmscaler.fit_transform(df_subset[['GarageArea']])


In [123]:
print (f'Valores de garagem normalizados, entre : vamor máximo {df_subset['GarageAreaNormalized'].max()} e valor minimo {df_subset['GarageAreaNormalized'].min()}  ')

Valores de garagem normalizados, entre : vamor máximo 3.0 e valor minimo -0.3333333333333333  


---

## End of Part 1

---

---

# Part 2

### Part 2 - 50% | Scope

Part 2 entails a project using the same dataset. The goal is to prove your data preprocessing skills. As output from this project, you should delivery the **notebook with the code you have done**.

Please apply, at least, **6 transformations** to the feature set you have in hands (or to the features which makes sense to apply the transformation). Each transformation should be accompanied by an explanation . Last but not least, compare the benefits of such transformation with the baseline score or the last best score. 

Regarding the variables you have to use throughout the Part 2, there are 6 in the total, 3 of them you are free to choose while the remaining 3 I have picked for you:
* YearBuilt
* LotFrontage
* MasVnrType

Make your baseline progressive, i.e. please consider the score from the previous transformation as the new baseline if it shows improvements. Example:

    Baseline - subset of transformations [None]  = 60% accuracy
    Iteration #1 - subset of transformations [A]     = 64% accuracy -> new baseline
    Iteration #2 - subset of transformations [A,B]   = 63% accuracy    (future scaling, ou imputação, ou, pelo menos 6 transformações)
    Iteration #3 - subset of transformations [A,B,C] = 68% accuracy -> new baseline
    ...
    Iteration #N - subset of transformations [A,B,C,..., N]
    (Being A, B, C a transformation that uses 1 or N features.)

Transformation example: encoding `color` & `country` with `One-Hot-Encoding`.

The `target` variable should be used to compute the accuracy (please use the `Target` you have created on the exercise 2.1, part1).


Remark:
* you shall use 6 variable for the assessment
* 3 out of 6 features are designated in the section on the top of the notebook
* the 3 remaining variables are up to you to choose
* you can consider any variable from the original dataset during this assessment

Above all, take this opportunity to practice :)

**Good luck!**